In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
import xarray as xr
import numpy as np
import pandas as pd

In [25]:
xr.set_options(keep_attrs=True)

In [26]:
import ecephys_analyses as ea 
from ecephys import utils
from ecephys.signal.xarray_utils import filter_dataset_by_hypnogram

In [27]:
def ds2df(ds):
    s = ds.to_dataframe().stack()
    s.index.names = s.index.names[:-1] + ['region']
    return s.to_frame(name='power')

In [28]:
subject = "Doppio"
experiment = "sleep-homeostasis"
bsl_condition = "recovery-sleep-circadian-match"
rec_condition = "recovery-sleep"
states = ["N1", "N2"]
duration = '02:00:00'
f_range = (0.5, 4)

In [29]:
rec_hyp = ea.load_hypnogram(subject, experiment, rec_condition)
rec_hyp = rec_hyp.keep_states(states).keep_first(duration)

rec_pow = ea.load_power(subject, experiment, rec_condition, "spg.nc").sel(frequency=slice(*f_range)).sum(dim='frequency')
rec_pow = filter_dataset_by_state(rec_pow, rec_hyp, states).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

In [30]:
bsl_hyp = ea.load_hypnogram(subject, experiment, bsl_condition)
bsl_hyp = bsl_hyp.keep_states(states).keep_between(rec_hyp.start_time.min().strftime("%H:%M:%S"), rec_hyp.start_time.max().strftime("%H:%M:%S"))

bsl_pow = ea.load_power(subject, experiment, bsl_condition, "spg.nc").sel(frequency=slice(*f_range)).sum(dim='frequency')
bsl_pow = filter_dataset_by_hypnogram(bsl_pow, bsl_hyp).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

In [31]:
nrem_pow = pd.concat([ds2df(bsl_pow).assign(condition="baseline"), ds2df(rec_pow).assign(condition="recovery")])

In [32]:
nrem_pow

power condition
datetime                      region                              
2020-03-17 15:11:43.999780779 mpta_wm_ref   49592.705167  baseline
                              sr_wm_ref    141458.991643  baseline
2020-03-17 15:11:46.999777763 mpta_wm_ref   14850.050950  baseline
                              sr_wm_ref    213550.733996  baseline
2020-03-17 15:11:49.999774746 mpta_wm_ref   17457.199927  baseline
...                                                  ...       ...
2020-03-18 17:57:45.996966108 sr_wm_ref    337275.270549  recovery
2020-03-18 17:57:48.996963091 mpta_wm_ref  649284.535997  recovery
                              sr_wm_ref    257836.165434  recovery
2020-03-18 17:57:51.996960074 mpta_wm_ref  381874.311244  recovery
                              sr_wm_ref    851230.021690  recovery

[8432 rows x 2 columns]

In [33]:
g = sns.displot(data=nrem_pow, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
nrem_pow_log = nrem_pow.copy()
nrem_pow_log['power'] = np.log(nrem_pow.power)

g = sns.displot(data=nrem_pow_log, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
nrem_pow_sqrt = nrem_pow.copy()
nrem_pow_sqrt['power'] = np.sqrt(nrem_pow.power)

g = sns.displot(data=nrem_pow_sqrt, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
def difference_of_means(df):
    means = df.groupby(['condition', 'region']).mean()
    return means.xs('recovery') - means.xs('baseline')

In [37]:
observed = difference_of_means(nrem_pow)

n_permutations = 10000
permutations = list()
permuted = nrem_pow.copy()
for i in range(n_permutations):
    np.random.shuffle(permuted.condition.values)
    permutations.append(difference_of_means(permuted))
    
df = pd.concat(permutations)

In [38]:
g = sns.displot(data=df, x='power', col='region', kind='hist')
g.set(xlabel='Difference of means (SD - BL)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
cx = df.xs('mpta_wm_ref').sort_values('power')
hc = df.xs('sr_wm_ref').sort_values('power')

In [40]:
pcx = np.sum(cx.power.values >= observed.xs('mpta_wm_ref').values)

In [41]:
phc = np.sum(hc.power.values >= observed.xs('sr_wm_ref').values)

In [42]:
print(f"Subject: {subject}")
print('Observed differences, Mean SD - Mean BSL:\n')
print(observed.to_markdown())
print('\n')
print('Shuffled differences >= observed:')
print(f"Cortex: {1 + pcx} / {n_permutations}")
print(f"Hippocampus: {1 + phc} / {n_permutations}")

Subject: Doppio
Observed differences, Mean SD - Mean BSL:

| region      |    power |
|:------------|---------:|
| mpta_wm_ref | 346453   |
| sr_wm_ref   |  99331.2 |


Shuffled differences >= observed:
Cortex: 1 / 10000
Hippocampus: 1 / 10000
